In [4]:
%pip install natsort
%pip install openpyxl

Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]
Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd 
import os
import glob
import difflib
from natsort import natsorted


In [6]:
n=10

mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/AJ/AJ {n}.xlsx')
excel_file = pd.ExcelFile(f'/workspaces/vikrant_dubai/RAW/AJ/AJ {n}.xlsx')

df = pd.DataFrame(columns=["Station Name", "Latitude", "Longitude", "Region", "Area Classification", "Station Operation Year"])
dfs = []
for i in range(len(excel_file.sheet_names)):

    df = pd.read_excel(f"/workspaces/vikrant_dubai/RAW/AJ/AJ {n}.xlsx", sheet_name = excel_file.sheet_names[i])

    df=df.drop([0,1,2,3,4,5])
    df=df.iloc[:,:8]
    df.head(2)

    mapped_data = {
        "Station Name": df.iloc[0,1],  
        "Latitude": df.iloc[1, 2],  
        "Longitude": df.iloc[1,4],  
        "Region": df.iloc[0, 4],  
        "Area Classification": df.iloc[0, 7], 
        "Station Operation Year": df.iloc[1, 7]   
    }
    temp_df = pd.DataFrame([mapped_data])
    dfs.append(temp_df)

final_df = pd.concat(dfs, ignore_index=True)
final_df.to_csv(f'/workspaces/vikrant_dubai/SQL/STdetails.csv', index=False)

In [7]:
for mon in range(1, 13):
    n=mon 

    mdf=pd.read_excel(f'/workspaces/vikrant_dubai/RAW/AJ/AJ {n}.xlsx')
    excel_file = pd.ExcelFile(f'/workspaces/vikrant_dubai/RAW/AJ/AJ {n}.xlsx')

    columns_of_main_df = ['Station Name','Date', 'Time', 'SO2', 'Data Cap. (SO2)', 'SO2 Avg', 'NO2', 'Data Cap. (NO2)', 'NO2 Avg',      'O3', 'Data Cap. (O3)', 'O3 8H Avg.', 'Daily Capture O3', 'O3 Daily Average', 'O3 Daily Average (NS)', 'CO',  'Data Cap. (CO)', 'CO 8H Avg.', 'Daily Capture CO', 'CO Daily Average', 'PM10',    'Data Cap. (PM10)', 'PM10 Avg', 'PM2.5', 'Data Cap. (PM2.5)', 'PM2.5 Avg', 'Lower Ambient Temperature', 'Upper Ambient Temperature', 'Barometric Pressure', 'Relative Humidity', 'Wind Direction', 'Wind Speed', 'H2S', 'Toluene', 'O-Xylene', 'Ethylbenzene', 'MP-Xylene', 'Benzene', 'CH4', 'NMHC', 'THC', 'Noise', 'Solar Radiation','Atm P.']
    for s in range(len(excel_file.sheet_names)):
        main_df = pd.DataFrame(columns=columns_of_main_df)

        # Read the Excel file and preprocess the data
        mdf = pd.read_excel(f'/workspaces/vikrant_dubai/RAW/AJ/AJ {n}.xlsx', sheet_name=excel_file.sheet_names[s])
        mdf = mdf.drop([0, 1, 2, 3,4,5,6,7,8,9]).reset_index(drop=True)
        mdf.columns = mdf.iloc[0]
        mdf = mdf[1:].reset_index(drop=True).drop([0, 1]).reset_index(drop=True)


        # Replace invalid 'Date/Time' entries with NaN
        mdf['Date/Time'] = pd.to_datetime(mdf['Date/Time'], format='%d/%m/%Y %H:%M', errors='coerce')

        # Convert 'Date/Time' column to separate 'Date' and 'Time' columns
        mdf['Date/Time'] = pd.to_datetime(mdf['Date/Time'], format='%d/%m/%Y %H:%M', dayfirst=True)
        mdf['Date'] = mdf['Date/Time'].dt.date
        mdf['Time'] = mdf['Date/Time'].dt.time
        new_df_columns = mdf.columns.tolist()

        # Iterate over the series and replace the column names
        for col_index in range(1, len(new_df_columns)):
            if 'DATA CAP.' in new_df_columns[col_index]:
                if new_df_columns[col_index-1] == 'DATA COUNT':
                    new_df_columns[col_index] = f"DATA CAP. ({new_df_columns[col_index-2]})"
                else:
                    new_df_columns[col_index] = f"DATA CAP. ({new_df_columns[col_index-1]})"

        mdf.columns = new_df_columns

        # Reorder columns and drop the original 'Date/Time' column
        mdf = mdf[['Date', 'Time'] + [col for col in mdf.columns if col not in ['Date', 'Time']]]
        mdf = mdf.drop(columns=['Date/Time'])

        # add column station name
        mdf['Station Name'] = excel_file.sheet_names[s]

        # Format the 'Date' column
        mdf['Date'] = pd.to_datetime(mdf['Date']).dt.strftime('%d/%m/%Y')

        # List of values to find matches for
        values = [col.upper() for col in mdf.columns]

        # List of column names
        l1 = [col.upper() for col in main_df.columns]

        # Loop through values and find the closest match
        matches = {}
        for value in values:
            closest_match = difflib.get_close_matches(value, l1, n=1)
            if closest_match:
                main_df[main_df.columns[main_df.columns.str.upper() == closest_match[0]][0]] = mdf[mdf.columns[mdf.columns.str.upper() == value][0]]
            else:
                main_df[value] = ''


            main_df.to_csv(f'/workspaces/vikrant_dubai/Transformed/{excel_file.sheet_names[s]}.csv', index=False)

    


            main_df.to_csv(f'/workspaces/vikrant_dubai/Transformed/{excel_file.sheet_names[s]}.csv', index=False)
        

    # Get a list of all CSV files in the transformed folder
    csv_files = [f for f in os.listdir('/workspaces/vikrant_dubai/Transformed') if f.endswith('.csv')]

    # Read and concatenate all CSV files
    combined_df = pd.concat([pd.read_csv(os.path.join('/workspaces/vikrant_dubai/Transformed', f)) for f in csv_files], ignore_index=True)

    # Save the combined DataFrame to a new CSV file
    combined_df.to_csv(f'/workspaces/vikrant_dubai/final/AJ/AJ {n}.csv', index=False)


    # Get a list of all files in the Transformed folder
    files = glob.glob('/workspaces/vikrant_dubai/Transformed/*')

    # Iterate over the list of files and remove each one
    for f in files:
        os.remove(f)


OSError: Cannot save file into a non-existent directory: '/workspaces/vikrant_dubai/Transformed'

In [3]:
# Get a list of all CSV files in the final/AJ folder
final_aj_csv_files = [f for f in os.listdir('/workspaces/vikrant_dubai/final/AJ') if f.endswith('.csv')]
final_aj_csv_files = natsorted(final_aj_csv_files)

# Read and concatenate all CSV files
combined_final_aj_df = pd.concat([pd.read_csv(os.path.join('/workspaces/vikrant_dubai/final/AJ', f)) for f in final_aj_csv_files], ignore_index=True)

# Save the combined DataFrame to a new CSV file
combined_final_aj_df.to_csv('/workspaces/vikrant_dubai/final/comb/AJ_combined.csv', index=False)

# Remove all files in the final/AJ folder
for f in final_aj_csv_files:
    os.remove(os.path.join('/workspaces/vikrant_dubai/final/AJ', f))